In [2]:
import duckdb
from tqdm import tqdm
import pandas as pd

In [3]:
con = duckdb.connect("../data/readmes.db")

In [4]:
df = con.execute("SELECT DISTINCT * FROM readmes WHERE readme != '' ").fetchdf()

In [44]:
ds_names = con.execute("SELECT id, name FROM datasets").fetchdf()

In [46]:
ds_names = ds_names.drop_duplicates(subset="name")

In [ ]:
mentions = []

for _, row in tqdm(df.iterrows()):
    mentioned = []
    rmd = row.readme.lower()
    for _, n in ds_names.iterrows():
        if n["name"].lower() in rmd:
            mentioned.append(n.id)
    if "cc-main" in rmd or "common crawl" in rmd or "commoncrawl" in rmd:
        mentioned.append("commoncrawl")
    mentions.append(mentioned)

18374it [1:55:33,  2.65it/s]


In [58]:
df["mentioned"] = mentions
df.to_parquet("../data/mentions.parquet")
df.to_csv("../data/mentions.csv")

In [66]:
df["mentioned_cc"] = df.readme.str.lower().str.contains("commoncrawl") | df.readme.str.lower().str.contains("common crawl") | df.readme.str.lower().str.contains("cc-main")

In [72]:
df[df["mentioned_cc"]][["id"]].to_csv("../data/mentions_cc.csv", index=False)

# Processed

In [8]:
df = pd.read_parquet("../data/mentions.parquet")

In [9]:
cc_filtered = pd.read_csv("../data/mentions_cc_filtered.csv")

In [10]:
cc_filtered

,id
0,tiiuae/falcon-refinedweb
1,cis-lmu/GlotCC-V1
2,PleIAs/common_corpus
3,erhwenkuo/c4-chinese-zhtw
4,eduagarcia/CrawlPT_dedup
5,HuggingFaceFW/fineweb-2
6,togethercomputer/RedPajama-Data-1T-Sample
7,ontocord/CulturaY
8,mlfoundations/dclm-baseline-1.0-parquet
9,ontocord/fineweb-permissive-multilingual-2m


In [26]:
# Find datasets that mention CC datasets but aren't in the CC filtered list themselves
# and also aren't already in the mentions_cc.csv file
cc_mentions = pd.read_csv("../data/mentions_cc.csv")["id"]
filtered_df = df[
    df.mentioned.apply(lambda x: any(cc_id in x for cc_id in cc_filtered["id"])) & 
    ~df["id"].isin(cc_filtered["id"]) & 
    ~df["id"].isin(cc_mentions)
][["id"]]

filtered_df.to_csv("../data/mentions_depth_1.csv", index=False)